# 4. Inference

In [1]:
import sys, os
sys.path.append("../../..")

import pathSetting
import pandas as pd

from Clust.clust.ML.common.common import p1_integratedDataSaving as p1
from Clust.clust.ML.common.common import p2_dataSelection as p2
from Clust.clust.ML.common.common import p4_testing as p4
from Clust.clust.ML.regression import app

import torch
#import main_regression as mr
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"{device}" " is available.")

cpu is available.


## 4-1. (just for test) Data Preparation 

In [2]:
# 0. pick only one data
DataMeta = p1.read_json_data(pathSetting.DataMetaPath)
dataList =  list(DataMeta.keys())
# dataX

dataName_X = dataList[2]
dataSaveMode_X = DataMeta[dataName_X]["integrationInfo"]["DataSaveMode"]

# datay
dataName_y = dataList[3]
dataSaveMode_y = DataMeta[dataName_y]["integrationInfo"]["DataSaveMode"]
dataFolderName = "data_integrated_result"
current = os.getcwd()
dataFolderPath = os.path.join(current, dataFolderName)

window_num =144
data_X = p2.get_saved_integrated_data(dataSaveMode_X, dataName_X, dataFolderPath)[:window_num]

In [3]:
from Clust.clust.ingestion.mongo import mongoClient as mongo_client
from Clust.setting import influx_setting_KETI as ins

mongo_client = mongo_client.mongoClient(ins.CLUSTMetaInfo2)

db_name = 'model'
collection_name = 'meta'

all_model_meta = mongo_client.get_all_document(db_name, collection_name)

In [4]:
db_name = 'model'
collection_name = 'meta'
search = {'modelName': 'EnergyLSTMRegression'}
model_meta= mongo_client.get_document_by_json(db_name, collection_name, search)
model_meta = model_meta[1]

In [5]:
model_meta

{'author': 'KETI',
 'modelName': 'EnergyLSTMRegression',
 'modelVersion': 0.1,
 'trainDataInfo': {'startTime': '2016-01-11',
  'endTime': '2016-04-15',
  'dataInfo': [['life_indoor_environment', 'humidityTrain_10min'],
   ['life_indoor_environment', 'temperatureTrain_10min'],
   ['weather_outdoor_environment', 'belgiumChieverseAirportTrain_10min']],
  'processParam': {'refine_param': {'removeDuplication': {'flag': False},
    'staticFrequency': {'flag': False, 'frequency': None}},
   'outlier_param': {'certainErrorToNaN': {'flag': False},
    'unCertainErrorToNaN': {'flag': False, 'param': {}}},
   'imputation_param': {'flag': False,
    'imputation_method': [],
    'totalNonNanRatio': 80}},
  'integration_freq_sec': 600,
  'cleanParam': 'NoClean',
  'DataSaveMode': 'CSV'},
 'featureList': ['RH_1',
  'RH_2',
  'RH_3',
  'RH_4',
  'RH_5',
  'RH_6',
  'RH_7',
  'RH_8',
  'RH_9',
  'T1',
  'T2',
  'T3',
  'T4',
  'T5',
  'T6',
  'T7',
  'T8',
  'T9',
  'Press_mm_hg',
  'RH_out',
  'T_out'

In [6]:
#2
# from Clust.clust.transformation.type.NPArrayToDF import trans3NPtoDF
# inputXDF, y = trans3NPtoDF(inputX, inputy,'2000-1-1')

In [7]:
# 4. Testing
df_result = app.get_inference_result(data_X, model_meta, window_num)

['RH_1', 'RH_2', 'RH_3', 'RH_4', 'RH_5', 'RH_6', 'RH_7', 'RH_8', 'RH_9', 'T1', 'T2', 'T3', 'T4', 'T5', 'T6', 'T7', 'T8', 'T9', 'Press_mm_hg', 'RH_out', 'T_out', 'Tdewpoint', 'Visibility', 'Windspeed']
['value']

Start testing data

** Dimension of result for inference dataset = (1,)
[0.30163723]
<class 'numpy.ndarray'>
----------------- ['value']


In [8]:
type(df_result)

pandas.core.frame.DataFrame

In [9]:
df_result

,value
0,11.656283
